## 启动服务

In [1]:
import mag as mag
import time
import json
from IPython.display import display, Markdown

In [2]:
# 启动服务
mag.start()

MAG服务器已启动 (PID: 25636), 日志: C:\Users\keta\AppData\Local\Temp\mag_sdk_logs\mag_server.log


True

In [ ]:
# 关闭服务
mag.shutdown()

In [3]:
is_running = mag.is_running()
print(f"服务运行状态: {is_running}")

服务运行状态: True


## 模型管理

#### （1）新增模型

In [ ]:
new_model = {
    "name": "deepseek-chat",
    "base_url": "https://api.openai.com/v1",
    "api_key": "sk-your-api-key",
    "model": "deepseek-chat"
}
mag.add_model(new_model)

{'status': 'success', 'message': "模型 'deepseek-chat1' 添加成功"}

#### （2）列出模型

In [28]:
models = mag.list_models()
print(f"找到 {len(models)} 个模型")
for model in models:
    print(f"\n模型名称：{model['name']} \nBase_url：{model['base_url']}\n")

找到 3 个模型

模型名称：Qwen/Qwen2.5-32B-Instruct 
Base_url：https://api-inference.modelscope.cn/v1/


模型名称：deepseek-chat 
Base_url：https://gateway.ai.cloudflare.com/v1/79dc4e1b350a1ac66b63c0f6a9e0c7a6/deepseek/deepseek


模型名称：deepseek-chat1 
Base_url：https://api.openai.com/v1



#### （3）更新模型

In [ ]:
# 更新模型
updated_model = {
    "name": "deepseek-chat",
    "base_url": "https://api.openai.com/v1",
    "api_key": "sk-your-api-key",
    "model": "deepseek-chat"
}
result = mag.update_model("deepseek-chat1", updated_model)
print(result)

{'status': 'success', 'message': "模型 'deepseek-chat1' 更新成功"}


#### （4）删除模型

In [32]:
mag.delete_model("deepseek-chat1")

{'status': 'success', 'message': "模型 'deepseek-chat1' 删除成功"}

## MCP管理

#### （1）增加/删除server

In [ ]:
# 增加server
servers = {"mcpServers": {
    "memory": {
      "autoApprove": [],
      "timeout": 60,
      "command": "docker",
      "args": [
        "run",
        "-i",
        "-v",
        "claude-memory:/app/dist",
        "--rm",
        "mcp/memory"
      ],
      "transportType": "stdio"
    },
    "fetch": {
      "autoApprove": [
        "fetch"
      ],
      "timeout": 60,
      "command": "uvx",
      "args": [
        "mcp-server-fetch"
      ],
      "transportType": "stdio"
    }
}}
result = mag.add_server(servers=servers)
print(json.dumps(result, indent=4, ensure_ascii=False))

{
    "status": "success",
    "message": "成功添加 2 个服务器",
    "added_servers": [
        "memory",
        "fetch"
    ],
    "duplicate_servers": [],
    "skipped_servers": [],
    "update_result": {
        "status": {
            "message": "配置已更新并通知MCP Client"
        }
    }
}


In [ ]:
# 删除server
result = mag.remove_server(["fetch","memory"])
print(json.dumps(result, indent=4, ensure_ascii=False))

{
    "status": "success",
    "message": "成功删除 2 个服务器",
    "removed_servers": [
        "fetch",
        "memory"
    ],
    "not_found_servers": [],
    "total_requested": 2,
    "update_result": {
        "status": {
            "message": "配置已更新并通知MCP Client"
        }
    }
}


#### （2）更新server配置文件（适合全面更新）

#### （3）连接mcp server

In [23]:
# 连接单个
mag.connect_mcp("fetch")

{'status': 'connected', 'server': 'fetch', 'tools': ['fetch']}

In [12]:
# 连接所有
result = mag.connect_mcp("all")
print(json.dumps(result, indent=2, ensure_ascii=False))

{
  "status": "completed",
  "summary": {
    "total_servers": 4,
    "successful_connections": 4,
    "failed_connections": 0,
    "already_connected": 0
  },
  "servers": {
    "bilibili": {
      "status": "connected",
      "tools": [
        "general_search"
      ]
    },
    "arxiv": {
      "status": "connected",
      "tools": [
        "search_arxiv"
      ]
    },
    "memory": {
      "status": "connected",
      "tools": [
        "create_entities",
        "create_relations",
        "add_observations",
        "delete_entities",
        "delete_observations",
        "delete_relations",
        "read_graph",
        "search_nodes",
        "open_nodes"
      ]
    },
    "fetch": {
      "status": "connected",
      "tools": [
        "fetch"
      ]
    }
  },
  "tools": {
    "bilibili": [
      "general_search"
    ],
    "arxiv": [
      "search_arxiv"
    ],
    "memory": [
      "create_entities",
      "create_relations",
      "add_observations",
      "delete_en

In [ ]:
# 更新mcp server
# 此举将会替代之前的配置文件
config={
  "mcpServers": {
    "memory": {
      "autoApprove": [],
      "timeout": 60,
      "command": "docker",
      "args": [
        "run",
        "-i",
        "-v",
        "claude-memory:/app/dist",
        "--rm",
        "mcp/memory"
      ],
      "transportType": "stdio",
    },
    "fetch": {
      "autoApprove": [
        "fetch"
      ],
      "timeout": 60,
      "command": "uvx",
      "args": [
        "mcp-server-fetch"
      ],
      "transportType": "stdio",
    }
  }
}
result = mag.update_mcp_config(config=config)
print(json.dumps(result, indent=4, ensure_ascii=False))

{
    "status": {
        "message": "配置已更新并通知MCP Client"
    }
}


#### （4）获取MCP配置

In [27]:
mcp_config = mag.get_mcp_config()
print(json.dumps(mcp_config, indent=8, ensure_ascii=False))

{
        "mcpServers": {
                "memory": {
                        "autoApprove": [],
                        "disabled": false,
                        "timeout": 60,
                        "command": "docker",
                        "args": [
                                "run",
                                "-i",
                                "-v",
                                "claude-memory:/app/dist",
                                "--rm",
                                "mcp/memory"
                        ],
                        "transportType": "stdio"
                },
                "fetch": {
                        "autoApprove": [
                                "fetch"
                        ],
                        "disabled": false,
                        "timeout": 60,
                        "command": "uvx",
                        "args": [
                                "mcp-server-fetch"
                        ],
                 

#### （5）获取MCP状态/查看工具

In [40]:
mcp_status = mag.get_mcp_status()
for server, status in mcp_status.items():
    print(f"\n{server}: {'已连接' if status.get('connected') else '未连接'}")
    print(f"{server}工具：{status.get('tools')}")
print(json.dumps(mcp_status, indent=4, ensure_ascii=False))


memory: 已连接
memory工具：['create_entities', 'create_relations', 'add_observations', 'delete_entities', 'delete_observations', 'delete_relations', 'read_graph', 'search_nodes', 'open_nodes']

fetch: 已连接
fetch工具：['fetch']
{
    "memory": {
        "connected": true,
        "init_attempted": true,
        "tools": [
            "create_entities",
            "create_relations",
            "add_observations",
            "delete_entities",
            "delete_observations",
            "delete_relations",
            "read_graph",
            "search_nodes",
            "open_nodes"
        ],
        "error": null
    },
    "fetch": {
        "connected": true,
        "init_attempted": true,
        "tools": [
            "fetch"
        ],
        "error": null
    }
}


#### （6）获取mcp工具参数

In [36]:
mcp_tools = mag.get_tools()
print(json.dumps(mcp_tools, indent=2, ensure_ascii=False))

{
  "memory": [
    {
      "name": "create_entities",
      "description": "Create multiple new entities in the knowledge graph",
      "input_schema": {
        "type": "object",
        "properties": {
          "entities": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the entity"
                },
                "entityType": {
                  "type": "string",
                  "description": "The type of the entity"
                },
                "observations": {
                  "type": "array",
                  "items": {
                    "type": "string"
                  },
                  "description": "An array of observation contents associated with the entity"
                }
              },
              "required": [
                "name",
                "entityType",
     

## 图管理

#### （1）导入一个图

In [24]:
# 从包导入
# graph = mag.import_graph("test.zip")
# print(json.dumps(graph, indent=2, ensure_ascii=False))

# 从config导入
graph = mag.import_graph(r"news_processing_workflow.json")
print(json.dumps(graph, indent=2, ensure_ascii=False))

{
  "status": "success",
  "message": "图 'news_processing_workflow' 导入成功"
}


#### （2）导出一个图

In [10]:
graph = mag.export_graph("test")
print(json.dumps(graph, indent=2, ensure_ascii=False))

{
  "status": "success",
  "message": "图 'test' 导出成功",
  "file_path": "C:\\Users\\keta\\.mag\\exports\\test.zip"
}


#### （3）列出所有图

In [12]:
print("\n列出所有图:")
graphs = mag.list_graphs()
print(f"找到 {len(graphs)} 个图: {graphs}")


列出所有图:
找到 3 个图: ['Bilibili搜索循环测试', 'easy_search', 'news_report']


#### （4）查看一个图配置文件

In [3]:
result = mag.get_graph('Bilibili搜索循环测试')
print(json.dumps(result, indent=2, ensure_ascii=False))

{
  "name": "Bilibili搜索循环测试",
  "description": "使用循环功能从bilibili搜索内容并总结至少30条信息",
  "nodes": [
    {
      "name": "search_node",
      "is_subgraph": false,
      "model_name": "deepseek-chat",
      "mcp_servers": [
        "bilibili"
      ],
      "system_prompt": "你是一个专业的视频内容搜索助手。你需要使用bilibili搜索工具获取用户请求的视频信息。",
      "user_prompt": "请使用bilibili_search工具搜索关于用户提到的主题的视频。执行全面的搜索，获取最相关的视频信息。\n\n用户输入:\n{start}",
      "input_nodes": [
        "start"
      ],
      "output_nodes": [
        "summarize_node"
      ],
      "output_enabled": true,
      "is_start": true,
      "is_end": false,
      "subgraph_name": null,
      "position": null,
      "level": 0,
      "handoffs": null,
      "description": "使用bilibili搜索视频内容"
    },
    {
      "name": "summarize_node",
      "is_subgraph": false,
      "model_name": "deepseek-chat",
      "mcp_servers": [],
      "system_prompt": "你是一个专业的内容总结助手。你需要将搜索结果总结为清晰的条目列表。",
      "user_prompt": "请将以下搜索结果总结为一条一条的内容列表。每条内容包括视频标题、作者、发布时间、播放量和简短介绍。确保格

#### （5）绘图

In [5]:
graph_detail = mag.get_graph_detail("easy_search")

# 渲染README
if graph_detail['readme']:
    display(Markdown(graph_detail['readme']))
else:
    print("未找到README文件")

# easy_search
优化循环和全局输出功能的知识探索系统，添加了知识探索摘要节点

## 流程图
```mermaid
graph TD
    Input(["start"]);
    Input(["start"]) --> planning_node["planning_node"];
    planning_node["planning_node"] --> knowledge_exploration_node["knowledge_exploration_node"];
    knowledge_exploration_node["knowledge_exploration_node"] --> exploration_summary_node["exploration_summary_node"];
    exploration_summary_node["exploration_summary_node"] --> decision_node{"decision_node"};
    decision_node{"decision_node"} --> knowledge_exploration_node["knowledge_exploration_node"];
    decision_node{"decision_node"} --> integration_node["integration_node"];
    integration_node["integration_node"] --> answer_node["answer_node"];
    answer_node["answer_node"] --> Output(["end"]);
```

## 节点列表
### planning_node
- 描述: 规划知识探索路径
- 使用模型: deepseek-chat

### knowledge_exploration_node
- 描述: 深入探索知识点
- 使用模型: deepseek-chat
- MCP服务器: bilibili

### exploration_summary_node
- 描述: 总结已探索的知识点和探索进度
- 使用模型: deepseek-chat

### decision_node
- 描述: 判断是否需要继续探索知识点或生成最终答案
- 使用模型: deepseek-chat
- 最大决策次数: 5

### integration_node
- 描述: 整合所有探索的知识
- 使用模型: deepseek-chat

### answer_node
- 描述: 生成最终答案
- 使用模型: deepseek-chat
- 此节点为图的结束节点

## 使用的MCP服务器
### bilibili
- 状态: 启用
- 超时: 60秒
- 自动批准的工具: general_search

## 使用的模型
### deepseek-chat
- 基础URL: https://api.deepseek.com/v1
- 模型标识符: deepseek-chat


#### （6）将图导出为mcp server 脚本

In [15]:
mcp = mag.generate_mcp_script("Bilibili搜索循环测试")

# 保存并行执行脚本
with open("bilibili_search_parallel.py", "w", encoding="utf-8") as f:
    f.write(mcp["parallel_script"])
print("并行执行server脚本已保存到: bilibili_search_parallel.py")

# 保存顺序执行脚本
with open("bilibili_search_sequential.py", "w", encoding="utf-8") as f:
    f.write(mcp["sequential_script"])
print("顺序执行server脚本已保存到: bilibili_search_sequential.py")

并行执行server脚本已保存到: bilibili_search_parallel.py
顺序执行server脚本已保存到: bilibili_search_sequential.py


#### （7） 运行图

In [ ]:
result = mag.run("easy_search")

## 对话管理

#### (1)列出所有对话

In [12]:
result = mag.list_conversations()
result

['Bilibili搜索循环测试_20250510_141314',
 'Bilibili搜索循环测试_20250510_141741',
 'Bilibili搜索循环测试_20250510_142730',
 'Bilibili搜索循环测试_20250510_142927',
 'MCPAnalyzer_20250507_231830',
 'test_20250515_095446',
 'test_20250515_100721',
 'test_20250515_101006',
 'test_20250515_114559',
 'test_20250515_115106',
 'test_20250515_125713',
 '知识探索与答案生成系统_20250513_164434',
 '知识探索与答案生成系统_20250513_212830',
 '知识探索与答案生成系统_20250513_224142',
 '知识探索与答案生成系统_20250514_001449',
 '知识探索与答案生成系统_20250514_003112',
 '知识探索与答案生成系统_20250514_221727',
 '知识探索与答案生成系统_20250514_224847',
 '知识探索与答案生成系统_20250514_232249',
 '知识探索与答案生成系统_20250514_233341',
 '知识探索与答案生成系统_20250515_000116',
 '知识探索与答案生成系统_20250515_000221',
 '知识探索与答案生成系统_20250515_001345']

#### (2)获取对话结果

In [12]:
result = mag.get_conversation('test_20250515_125713')
display(Markdown(result.get("output")))

以下是两篇新闻中提取的关键实体及其关系，以表格形式呈现：

| **实体类型**  | **实体名称**               | **关系**                     | **关系对象**               | **备注**                     |
|----------------|----------------------------|------------------------------|----------------------------|------------------------------|
| 人物           | 沃伦·巴菲特                | 宣布退休                     | 伯克希尔·哈撒韦公司        | 两篇新闻共同提及             |
| 人物           | 格雷格·阿贝尔              | 接任CEO                      | 伯克希尔·哈撒韦公司        | 两篇新闻共同提及             |
| 组织           | 伯克希尔·哈撒韦公司        | 召开股东大会                 | 2025年股东大会             | 两篇新闻共同提及             |
| 组织           | 美国政府                   | 被批评贸易政策               | 沃伦·巴菲特                | 两篇新闻共同提及             |
| 概念           | 贸易政策                   | 被批评                       | 沃伦·巴菲特                | 两篇新闻共同提及             |
| 概念           | 财政赤字                   | 不可持续                     | 沃伦·巴菲特                | 第一篇新闻提及               |
| 概念           | 通胀                       | 非常严重                     | 沃伦·巴菲特                | 第一篇新闻提及               |
| 投资领域       | AI投资                     | 不会围绕投资                 | 沃伦·巴菲特                | 第一篇新闻提及               |
| 投资领域       | 日本投资                   | 不会卖掉股票                 | 伯克希尔·哈撒韦公司        | 第一篇新闻提及               |
| 投资领域       | 房地产与股票               | 房地产吸引力更低             | 沃伦·巴菲特                | 第一篇新闻提及               |
| 投资策略       | 全仓投资                   | 从未全仓投资                 | 沃伦·巴菲特                | 第一篇新闻提及               |
| 职业建议       | 选择喜欢的工作             | 建议                         | 沃伦·巴菲特                | 第一篇新闻提及               |
| 市场现象       | 美股波动                   | 应理性看待                   | 沃伦·巴菲特                | 第一篇新闻提及               |
| 科技公司       | 科技“七巨头”               | 观察资本密集程度             | 沃伦·巴菲特                | 第一篇新闻提及               |
| 能源领域       | 能源合作                   | 呼吁政府与私营企业合作       | 沃伦·巴菲特                | 第一篇新闻提及               |
| 财务数据       | 2025年第一季度运营利润     | 同比下降14%                  | 伯克希尔·哈撒韦公司        | 第二篇新闻提及               |
| 财务数据       | 保险承保利润               | 暴跌                         | 伯克希尔·哈撒韦公司        | 第二篇新闻提及               |
| 财务数据       | 美元贬值                   | 影响利润                     | 伯克希尔·哈撒韦公司        | 第二篇新闻提及               |
| 风险因素       | 关税政策                   | 可能影响利润                 | 伯克希尔·哈撒韦公司        | 第二篇新闻提及               |
| 风险因素       | 地缘政治风险               | 可能影响利润                 | 伯克希尔·哈撒韦公司        | 第二篇新闻提及               |
| 人物           | 格雷格·阿贝尔              | 被视为继任者                 | 沃伦·巴菲特                | 第二篇新闻提及               |
| 人物           | 格雷格·阿贝尔              | 投资者对其表现有待观察       | 现金投资                   | 第二篇新闻提及               |
| 资产处置       | 公司股份                   | 不会出售                     | 沃伦·巴菲特                | 第二篇新闻提及               |
| 资产处置       | 公司股份                   | 最终捐赠                     | 沃伦·巴菲特                | 第二篇新闻提及               |

### 备注：
1. **两篇新闻共同提及的实体**：沃伦·巴菲特、格雷格·阿贝尔、伯克希尔·哈撒韦公司、美国政府、贸易政策。
2. **关系标注**：表格中的“关系”列描述了实体之间的主要互动或属性。
3. **实体类型**：包括人物、组织、概念、投资领域、投资策略、职业建议、市场现象、科技公司、能源领域、财务数据、风险因素、资产处置等，以帮助分类和理解。

**60秒新闻播报稿：**  

【开场】这里是财经快讯，为您带来全球市场最新动态。  

【巴菲特退休与接班人】"股神"沃伦·巴菲特在伯克希尔股东大会上宣布，将于年底卸任CEO一职，由格雷格·阿贝尔接任。这一突然决定引发市场震动，但巴菲特强调不会出售任何公司股份，未来将悉数捐赠。  

【贸易政策批评】巴菲特直言美国政府将贸易作为武器是"重大错误"，关税扰乱全球市场，呼吁回归平衡贸易。  

【业绩与挑战】伯克希尔一季度运营利润骤降14%，主因保险业务下滑及美元贬值。公司警告，地缘风险或进一步冲击盈利。  

【投资观点】巴菲特重申对日本投资的长期承诺，但避谈AI领域；同时警示美国财政赤字"不可持续"，通胀问题严峻。  

【结语】这场标志性会议，既为巴菲特时代画下句点，也为全球经济格局留下深刻思考。本台将持续关注后续影响。  

【结束】以上是今日财经要闻，感谢收看。

<!DOCTYPE html>
<html lang="zh-CN">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>巴菲特2025股东大会精华 | 投资大师的谢幕演讲</title>
    <link href="https://cdn.staticfile.org/font-awesome/6.4.0/css/all.min.css" rel="stylesheet">
    <link href="https://cdn.staticfile.org/tailwindcss/2.2.19/tailwind.min.css" rel="stylesheet">
    <link href="https://fonts.googleapis.com/css2?family=Noto+Serif+SC:wght@400;500;600;700&family=Noto+Sans+SC:wght@300;400;500;700&display=swap" rel="stylesheet">
    <script src="https://cdn.jsdelivr.net/npm/mermaid@latest/dist/mermaid.min.js"></script>
    <style>
        :root {
            --primary: #2563eb;
            --secondary: #1e40af;
            --accent: #f59e0b;
            --text: #1f2937;
            --light-bg: #f9fafb;
            --dark-bg: #111827;
            --dark-text: #f3f4f6;
        }
        
        body {
            font-family: 'Noto Sans SC', 'Noto Serif SC', Tahoma, Arial, Roboto, "Droid Sans", "Helvetica Neue", "Droid Sans Fallback", "Heiti SC", "Hiragino Sans GB", Simsun, sans-serif;
            color: var(--text);
            background-color: var(--light-bg);
            transition: all 0.3s ease;
        }
        
        .dark {
            color: var(--dark-text);
            background-color: var(--dark-bg);
        }
        
        .dark .card {
            background-color: #1f2937;
            border-color: #374151;
        }
        
        .dark .hero-overlay {
            background-color: rgba(0, 0, 0, 0.7);
        }
        
        .hero {
            background-image: url('https://images.unsplash.com/photo-1611974789855-9c2a0a7236a3?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D&auto=format&fit=crop&w=2070&q=80');
            background-size: cover;
            background-position: center;
        }
        
        .card {
            transition: transform 0.3s ease, box-shadow 0.3s ease;
        }
        
        .card:hover {
            transform: translateY(-5px);
            box-shadow: 0 20px 25px -5px rgba(0, 0, 0, 0.1), 0 10px 10px -5px rgba(0, 0, 0, 0.04);
        }
        
        .quote::before {
            content: '"';
            font-size: 4rem;
            line-height: 1;
            color: var(--accent);
            opacity: 0.3;
            position: absolute;
            left: -1rem;
            top: -1rem;
        }
        
        .timeline-item::before {
            content: '';
            position: absolute;
            left: -1.5rem;
            top: 0;
            width: 1rem;
            height: 1rem;
            border-radius: 50%;
            background-color: var(--accent);
            border: 3px solid white;
            z-index: 1;
        }
        
        .timeline::before {
            content: '';
            position: absolute;
            left: 0.5rem;
            top: 0;
            bottom: 0;
            width: 2px;
            background-color: #e5e7eb;
        }
        
        .dark .timeline::before {
            background-color: #374151;
        }
        
        .drop-cap::first-letter {
            float: left;
            font-size: 4.5rem;
            line-height: 0.9;
            margin-right: 0.5rem;
            margin-top: 0.3rem;
            color: var(--accent);
            font-weight: bold;
        }
        
        .fade-in {
            animation: fadeIn 0.5s ease-in;
        }
        
        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(20px); }
            to { opacity: 1; transform: translateY(0); }
        }
    </style>
</head>
<body class="min-h-screen">
    <!-- Dark Mode Toggle -->
    <div class="fixed top-4 right-4 z-50">
        <button id="themeToggle" class="p-2 rounded-full bg-white dark:bg-gray-800 shadow-lg">
            <i class="fas fa-moon text-gray-800 dark:text-yellow-300"></i>
            <i class="fas fa-sun text-yellow-300 dark:hidden"></i>
        </button>
    </div>

    <!-- Hero Section -->
    <section class="hero relative h-96 md:h-screen/2 flex items-center justify-center text-white">
        <div class="hero-overlay absolute inset-0 bg-black opacity-50"></div>
        <div class="container mx-auto px-4 relative z-10 text-center">
            <h1 class="text-4xl md:text-6xl font-bold mb-4 animate-fade-in">巴菲特2025股东大会</h1>
            <h2 class="text-xl md:text-2xl font-semibold mb-8">投资大师的谢幕演讲与智慧结晶</h2>
            <div class="max-w-3xl mx-auto">
                <p class="text-lg md:text-xl mb-8">94岁的"股神"沃伦·巴菲特在伯克希尔·哈撒韦年度股东大会上宣布退休计划，并分享了他对全球经济、投资策略和人生智慧的深刻见解。</p>
                <a href="#content" class="inline-block bg-blue-600 hover:bg-blue-700 text-white font-bold py-3 px-6 rounded-full transition duration-300 transform hover:scale-105">
                    <i class="fas fa-arrow-down mr-2"></i>探索智慧
                </a>
            </div>
        </div>
    </section>

    <!-- Main Content -->
    <main id="content" class="container mx-auto px-4 py-12 md:py-16">
        <!-- Introduction -->
        <section class="mb-16 fade-in">
            <div class="max-w-4xl mx-auto">
                <p class="drop-cap text-lg md:text-xl leading-relaxed mb-8">2025年伯克希尔·哈撒韦股东大会标志着投资传奇沃伦·巴菲特时代的逐步落幕。94岁高龄的"股神"在这场备受瞩目的会议上宣布了退休计划，同时分享了他对全球经济、投资策略和人生智慧的深刻见解。本文全面梳理了大会的核心内容，带您领略这位投资大师的收官智慧。</p>
                
                <div class="bg-blue-50 dark:bg-gray-800 p-6 rounded-lg border-l-4 border-blue-500 mb-8">
                    <div class="flex items-start">
                        <div class="flex-shrink-0 mt-1">
                            <i class="fas fa-quote-left text-blue-500 text-2xl"></i>
                        </div>
                        <div class="ml-4">
                            <p class="text-lg font-medium text-gray-800 dark:text-gray-200">"投资不是关于智商的问题，而是关于性格的问题。"</p>
                            <p class="mt-2 text-gray-600 dark:text-gray-400">— 沃伦·巴菲特</p>
                        </div>
                    </div>
                </div>
            </div>
        </section>

        <!-- Key Points Grid -->
        <section class="mb-16 fade-in">
            <h2 class="text-3xl md:text-4xl font-bold mb-8 text-center">大会核心要点</h2>
            
            <div class="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-8">
                <!-- Retirement -->
                <div class="card bg-white dark:bg-gray-800 rounded-lg shadow-md overflow-hidden border border-gray-100 dark:border-gray-700">
                    <div class="p-6">
                        <div class="flex items-center mb-4">
                            <div class="bg-blue-100 dark:bg-blue-900 p-3 rounded-full mr-4">
                                <i class="fas fa-user-tie text-blue-600 dark:text-blue-300 text-xl"></i>
                            </div>
                            <h3 class="text-xl font-bold">退休与接班人</h3>
                        </div>
                        <ul class="space-y-3 text-gray-700 dark:text-gray-300">
                            <li class="flex items-start">
                                <i class="fas fa-check-circle text-green-500 mt-1 mr-2"></i>
                                <span>年底辞去CEO，由格雷格·阿贝尔接任</span>
                            </li>
                            <li class="flex items-start">
                                <i class="fas fa-check-circle text-green-500 mt-1 mr-2"></i>
                                <span>仍会提供帮助但决定权在阿贝尔手中</span>
                            </li>
                            <li class="flex items-start">
                                <i class="fas fa-check-circle text-green-500 mt-1 mr-2"></i>
                                <span>无意出售伯克希尔的任何股票</span>
                            </li>
                        </ul>
                    </div>
                </div>

                <!-- Trade -->
                <div class="card bg-white dark:bg-gray-800 rounded-lg shadow-md overflow-hidden border border-gray-100 dark:border-gray-700">
                    <div class="p-6">
                        <div class="flex items-center mb-4">
                            <div class="bg-yellow-100 dark:bg-yellow-900 p-3 rounded-full mr-4">
                                <i class="fas fa-globe text-yellow-600 dark:text-yellow-300 text-xl"></i>
                            </div>
                            <h3 class="text-xl font-bold">贸易与关税</h3>
                        </div>
                        <ul class="space-y-3 text-gray-700 dark:text-gray-300">
                            <li class="flex items-start">
                                <i class="fas fa-check-circle text-green-500 mt-1 mr-2"></i>
                                <span>贸易不应被当作武器</span>
                            </li>
                            <li class="flex items-start">
                                <i class="fas fa-check-circle text-green-500 mt-1 mr-2"></i>
                                <span>美国应寻求平衡的国际贸易</span>
                            </li>
                            <li class="flex items-start">
                                <i class="fas fa-check-circle text-green-500 mt-1 mr-2"></i>
                                <span>批评关税政策扰乱全球市场</span>
                            </li>
                        </ul>
                    </div>
                </div>

                <!-- Investment -->
                <div class="card bg-white dark:bg-gray-800 rounded-lg shadow-md overflow-hidden border border-gray-100 dark:border-gray-700">
                    <div class="p-6">
                        <div class="flex items-center mb-4">
                            <div class="bg-green-100 dark:bg-green-900 p-3 rounded-full mr-4">
                                <i class="fas fa-chart-line text-green-600 dark:text-green-300 text-xl"></i>
                            </div>
                            <h3 class="text-xl font-bold">投资观点</h3>
                        </div>
                        <ul class="space-y-3 text-gray-700 dark:text-gray-300">
                            <li class="flex items-start">
                                <i class="fas fa-check-circle text-green-500 mt-1 mr-2"></i>
                                <span>不会围绕AI进行投资</span>
                            </li>
                            <li class="flex items-start">
                                <i class="fas fa-check-circle text-green-500 mt-1 mr-2"></i>
                                <span>日本投资未来十年不会出售</span>
                            </li>
                            <li class="flex items-start">
                                <i class="fas fa-check-circle text-green-500 mt-1 mr-2"></i>
                                <span>房地产比股票更耗时且吸引力低</span>
                            </li>
                        </ul>
                    </div>
                </div>

                <!-- Finance -->
                <div class="card bg-white dark:bg-gray-800 rounded-lg shadow-md overflow-hidden border border-gray-100 dark:border-gray-700">
                    <div class="p-6">
                        <div class="flex items-center mb-4">
                            <div class="bg-red-100 dark:bg-red-900 p-3 rounded-full mr-4">
                                <i class="fas fa-money-bill-wave text-red-600 dark:text-red-300 text-xl"></i>
                            </div>
                            <h3 class="text-xl font-bold">财政赤字与通胀</h3>
                        </div>
                        <ul class="space-y-3 text-gray-700 dark:text-gray-300">
                            <li class="flex items-start">
                                <i class="fas fa-check-circle text-green-500 mt-1 mr-2"></i>
                                <span>美国财政赤字"不可持续"</span>
                            </li>
                            <li class="flex items-start">
                                <i class="fas fa-check-circle text-green-500 mt-1 mr-2"></i>
                                <span>长期未真正解决财政问题</span>
                            </li>
                            <li class="flex items-start">
                                <i class="fas fa-check-circle text-green-500 mt-1 mr-2"></i>
                                <span>通胀问题非常严重</span>
                            </li>
                        </ul>
                    </div>
                </div>

                <!-- Career Advice -->
                <div class="card bg-white dark:bg-gray-800 rounded-lg shadow-md overflow-hidden border border-gray-100 dark:border-gray-700">
                    <div class="p-6">
                        <div class="flex items-center mb-4">
                            <div class="bg-purple-100 dark:bg-purple-900 p-3 rounded-full mr-4">
                                <i class="fas fa-briefcase text-purple-600 dark:text-purple-300 text-xl"></i>
                            </div>
                            <h3 class="text-xl font-bold">职业建议</h3>
                        </div>
                        <ul class="space-y-3 text-gray-700 dark:text-gray-300">
                            <li class="flex items-start">
                                <i class="fas fa-check-circle text-green-500 mt-1 mr-2"></i>
                                <span>选择自己喜欢的工作</span>
                            </li>
                            <li class="flex items-start">
                                <i class="fas fa-check-circle text-green-500 mt-1 mr-2"></i>
                                <span>不要太在意初始薪水</span>
                            </li>
                            <li class="flex items-start">
                                <i class="fas fa-check-circle text-green-500 mt-1 mr-2"></i>
                                <span>兴趣和热情更重要</span>
                            </li>
                        </ul>
                    </div>
                </div>

                <!-- Energy -->
                <div class="card bg-white dark:bg-gray-800 rounded-lg shadow-md overflow-hidden border border-gray-100 dark:border-gray-700">
                    <div class="p-6">
                        <div class="flex items-center mb-4">
                            <div class="bg-indigo-100 dark:bg-indigo-900 p-3 rounded-full mr-4">
                                <i class="fas fa-bolt text-indigo-600 dark:text-indigo-300 text-xl"></i>
                            </div>
                            <h3 class="text-xl font-bold">能源合作</h3>
                        </div>
                        <ul class="space-y-3 text-gray-700 dark:text-gray-300">
                            <li class="flex items-start">
                                <i class="fas fa-check-circle text-green-500 mt-1 mr-2"></i>
                                <span>呼吁政府与私营企业合作</span>
                            </li>
                            <li class="flex items-start">
                                <i class="fas fa-check-circle text-green-500 mt-1 mr-2"></i>
                                <span>伯克希尔愿提供资本和知识</span>
                            </li>
                            <li class="flex items-start">
                                <i class="fas fa-check-circle text-green-500 mt-1 mr-2"></i>
                                <span>强调能源领域的重要性</span>
                            </li>
                        </ul>
                    </div>
                </div>
            </div>
        </section>

        <!-- Detailed Analysis -->
        <section class="mb-16 fade-in">
            <h2 class="text-3xl md:text-4xl font-bold mb-8 text-center">深入分析</h2>
            
            <div class="max-w-4xl mx-auto">
                <!-- Retirement Details -->
                <div class="mb-12">
                    <h3 class="text-2xl font-bold mb-4 flex items-center">
                        <i class="fas fa-user-tie text-blue-600 mr-3"></i>
                        退休与接班人计划
                    </h3>
                    <div class="bg-white dark:bg-gray-800 rounded-lg shadow-md p-6">
                        <p class="mb-4 text-gray-700 dark:text-gray-300">在2025年伯克希尔·哈撒韦股东大会上，94岁的沃伦·巴菲特宣布将于年底辞去公司CEO一职，这一决定震惊了投资界。巴菲特此前一直表示没有退休计划，但这次他明确表示将由非保险业务副董事长格雷格·阿贝尔接任首席执行官。</p>
                        
                        <div class="flex flex-col md:flex-row gap-6 mb-4">
                            <div class="flex-1">
                                <h4 class="font-semibold mb-2 text-blue-600 dark:text-blue-400">格雷格·阿贝尔简介</h4>
                                <ul class="list-disc pl-5 space-y-2 text-gray-700 dark:text-gray-300">
                                    <li>1962年出生于加拿大阿尔伯塔省</li>
                                    <li>1992年加入伯克希尔旗下能源公司</li>
                                    <li>2018年晋升为伯克希尔非保险业务副董事长</li>
                                    <li>以运营能力和资本配置能力著称</li>
                                </ul>
                            </

In [14]:
attactments = result.get("attachments")
print(json.dumps(attactments, indent=2, ensure_ascii=False))

[
  {
    "name": "summary_130204.html",
    "path": "C:\\Users\\keta\\.mag\\conversation\\test_20250515_125713\\attachment\\summary_130204.html",
    "size": 18492,
    "type": "html",
    "modified": "Thu May 15 13:02:04 2025",
    "relative_path": "attachment/summary_130204.html"
  },
  {
    "name": "speech_125909.docx",
    "path": "C:\\Users\\keta\\.mag\\conversation\\test_20250515_125713\\attachment\\speech_125909.docx",
    "size": 1089,
    "type": "docx",
    "modified": "Thu May 15 12:59:09 2025",
    "relative_path": "attachment/speech_125909.docx"
  },
  {
    "name": "sum_125854.md",
    "path": "C:\\Users\\keta\\.mag\\conversation\\test_20250515_125713\\attachment\\sum_125854.md",
    "size": 4910,
    "type": "md",
    "modified": "Thu May 15 12:58:54 2025",
    "relative_path": "attachment/sum_125854.md"
  }
]


In [19]:
import os
import webbrowser
import subprocess
from IPython.display import Markdown, HTML, display
import markdown
from docx import Document

def display_attachments(attachments):
    """
    显示附件内容，支持md、html文件,这里只提供demo，运行时产生其他扩展文件请自行设计。或直接前往目录查看。
    使用notebook的markdown进行展示
    
    参数:
    attachments - 附件列表
    """
    if not attachments:
        print("没有可显示的附件")
        return
    
    for attachment in attachments:
        file_type = attachment["type"].lower()
        file_path = attachment["path"]
        file_name = attachment["name"]
        
        # 跳过不支持的文件类型
        if file_type not in ["md", "html", "docx", "pdf"]:
            continue
            
        print(f"\n📄 {file_name} ({file_type.upper()}):")
        
        try:
            # Markdown文件 - 使用IPython的Markdown显示
            if file_type == "md":
                with open(file_path, 'r', encoding='utf-8') as f:
                    md_content = f.read()
                display(Markdown(md_content))
                
            # HTML文件 - 使用IPython的HTML显示，或在浏览器中打开
            elif file_type == "html":
                try:
                    webbrowser.open(file_path)
                except:
                    print(f"已在浏览器中打开 HTML 文件")
                
        except Exception as e:
            print(f"无法显示文件 {file_name}: {str(e)}")

display_attachments(attactments)


📄 summary_130204.html (HTML):

📄 speech_125909.docx (DOCX):

📄 sum_125854.md (MD):


以下是两篇新闻中提取的关键实体及其关系，以表格形式呈现：

| **实体类型**  | **实体名称**               | **关系**                     | **关系对象**               | **备注**                     |
|----------------|----------------------------|------------------------------|----------------------------|------------------------------|
| 人物           | 沃伦·巴菲特                | 宣布退休                     | 伯克希尔·哈撒韦公司        | 两篇新闻共同提及             |
| 人物           | 格雷格·阿贝尔              | 接任CEO                      | 伯克希尔·哈撒韦公司        | 两篇新闻共同提及             |
| 组织           | 伯克希尔·哈撒韦公司        | 召开股东大会                 | 2025年股东大会             | 两篇新闻共同提及             |
| 组织           | 美国政府                   | 被批评贸易政策               | 沃伦·巴菲特                | 两篇新闻共同提及             |
| 概念           | 贸易政策                   | 被批评                       | 沃伦·巴菲特                | 两篇新闻共同提及             |
| 概念           | 财政赤字                   | 不可持续                     | 沃伦·巴菲特                | 第一篇新闻提及               |
| 概念           | 通胀                       | 非常严重                     | 沃伦·巴菲特                | 第一篇新闻提及               |
| 投资领域       | AI投资                     | 不会围绕投资                 | 沃伦·巴菲特                | 第一篇新闻提及               |
| 投资领域       | 日本投资                   | 不会卖掉股票                 | 伯克希尔·哈撒韦公司        | 第一篇新闻提及               |
| 投资领域       | 房地产与股票               | 房地产吸引力更低             | 沃伦·巴菲特                | 第一篇新闻提及               |
| 投资策略       | 全仓投资                   | 从未全仓投资                 | 沃伦·巴菲特                | 第一篇新闻提及               |
| 职业建议       | 选择喜欢的工作             | 建议                         | 沃伦·巴菲特                | 第一篇新闻提及               |
| 市场现象       | 美股波动                   | 应理性看待                   | 沃伦·巴菲特                | 第一篇新闻提及               |
| 科技公司       | 科技“七巨头”               | 观察资本密集程度             | 沃伦·巴菲特                | 第一篇新闻提及               |
| 能源领域       | 能源合作                   | 呼吁政府与私营企业合作       | 沃伦·巴菲特                | 第一篇新闻提及               |
| 财务数据       | 2025年第一季度运营利润     | 同比下降14%                  | 伯克希尔·哈撒韦公司        | 第二篇新闻提及               |
| 财务数据       | 保险承保利润               | 暴跌                         | 伯克希尔·哈撒韦公司        | 第二篇新闻提及               |
| 财务数据       | 美元贬值                   | 影响利润                     | 伯克希尔·哈撒韦公司        | 第二篇新闻提及               |
| 风险因素       | 关税政策                   | 可能影响利润                 | 伯克希尔·哈撒韦公司        | 第二篇新闻提及               |
| 风险因素       | 地缘政治风险               | 可能影响利润                 | 伯克希尔·哈撒韦公司        | 第二篇新闻提及               |
| 人物           | 格雷格·阿贝尔              | 被视为继任者                 | 沃伦·巴菲特                | 第二篇新闻提及               |
| 人物           | 格雷格·阿贝尔              | 投资者对其表现有待观察       | 现金投资                   | 第二篇新闻提及               |
| 资产处置       | 公司股份                   | 不会出售                     | 沃伦·巴菲特                | 第二篇新闻提及               |
| 资产处置       | 公司股份                   | 最终捐赠                     | 沃伦·巴菲特                | 第二篇新闻提及               |

### 备注：
1. **两篇新闻共同提及的实体**：沃伦·巴菲特、格雷格·阿贝尔、伯克希尔·哈撒韦公司、美国政府、贸易政策。
2. **关系标注**：表格中的“关系”列描述了实体之间的主要互动或属性。
3. **实体类型**：包括人物、组织、概念、投资领域、投资策略、职业建议、市场现象、科技公司、能源领域、财务数据、风险因素、资产处置等，以帮助分类和理解。